## Data Loading

В наборе данных также присутствует дополнительная информация о треках: длительность, контентные embedding (анализ в другом блокноте), артист, альбом. Сейчас эти данные находятся в разных источниках и представляют собой разрозненные таблицы. Чтобы упростить дальнейшую обработку и анализ, объединим их в единый, целостный датасет, где каждая запись будет содержать всю необходимую информацию о треке в одном месте.

In [2]:

import pandas as  pd

track_length_seconds = pd.read_parquet("multi_event.parquet", engine="fastparquet")[["item_id","track_length_seconds"]].drop_duplicates()
artist = pd.read_parquet("artist_item_mapping.parquet", engine="fastparquet")
albums = pd.read_parquet("album_item_mapping.parquet", engine="fastparquet")

items_meta = (
    track_length_seconds
    .merge(artist, on=['item_id'], how='outer')
    .merge(albums, on=['item_id'], how='outer')
    .dropna() 
)

items_meta.head()


,item_id,track_length_seconds,artist_id,album_id
0,8326270,170.0,714356.0,1060653.0
1,8326270,170.0,714356.0,1486663.0
4,1441281,105.0,1043457.0,547645.0
6,286361,185.0,94924.0,141751.0
8,732449,240.0,94924.0,1632076.0


In [6]:
items_meta.drop_duplicates()

,item_id,track_length_seconds,artist_id,album_id
0,8326270,170.0,714356.0,1060653.0
1,8326270,170.0,714356.0,1486663.0
4,1441281,105.0,1043457.0,547645.0
6,286361,185.0,94924.0,141751.0
8,732449,240.0,94924.0,1632076.0
...,...,...,...,...
1793976,6222585,195.0,1117260.0,1604865.0
1793977,3922773,225.0,1117260.0,1604865.0
1793978,7215687,245.0,1084004.0,161282.0
1793979,7882581,400.0,52972.0,495062.0


Проверим что item_id и track_length_seconds соотносятся уникально. Таблица пустая - все пары уникальные track_length_seconds берем первый

In [21]:

dups = (
    items_meta.groupby("item_id")["track_length_seconds"]
    .nunique()
    .reset_index(name="uniq_lengths")
)

# item_id с несколькими track_length_seconds
problem_items = dups[dups["uniq_lengths"] > 1]
problem_items.head()


,item_id,uniq_lengths


In [19]:
items_meta.to_parquet("items_meta.parquet", index=False)